In [209]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm

In [ ]:
from geopy.geocoders import Nominatim
# geolocalização
geolocator = Nominatim(user_agent="myApp", timeout=5)

In [2]:
with open('data/houses.pkl', 'rb') as f:
    obj = pickle.load(f)

In [119]:
df = pd.DataFrame(obj.values())

Ajustando o valor do imóvel

In [120]:
df = pd.concat([df, pd.DataFrame(df.price.to_dict().values(), columns=['rent', 'Condomínio', 'Preço c/ condomínio','IPTU'])], axis=1)

Ajustando o banheiro, removendo a lista

In [121]:
df.bathrooms = df.bathrooms.map(lambda x: int(x[0]) if isinstance(x, list) and (not not x) else 0)

In [122]:
df.iloc[0:3].address.apply(geolocator.geocode).apply(lambda x: pd.Series([x.latitude, x.longitude], index=['location_lat', 'location_long']))

,location_lat,location_long
0,-23.650586,-46.718644
1,-23.723508,-46.693460
2,-21.132831,-51.102942


In [123]:
df.drop(columns=['total_price', 'area', 'description', 'transport', 'publisher_name', 'code', 'type'], inplace=True)

In [129]:
df['Condomínio'] = df['Condomínio'].str.replace('R$ ', '', regex=False)

In [134]:
df['Condomínio'] = df['Condomínio'].str.strip()

In [161]:
df['Condomínio'] = df['Condomínio'].replace({'Não informado': 0, 'Preço c/ condomínio':0, 'IPTU':'0'})

In [162]:
df['Condomínio'] = df['Condomínio'].str.replace('.', '', regex=False)

In [163]:
df['Condomínio'] = df['Condomínio'].fillna(0)

In [165]:
df['Condomínio'] = df['Condomínio'].astype(int)

In [169]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].replace({'R$ ':''}, regex=False).fillna(0)

In [185]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).fillna(0).astype(int)

In [194]:
df['IPTU'] = df['IPTU'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).replace({' Não informado ':0}, regex=False).fillna(0).astype(int)

In [196]:
df.drop(columns=['price'], inplace=True)

In [198]:
df.head()

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
0,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,1.0,"[Piscina, Churrasqueira, Condomínio fechado, A...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,225
1,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,0
2,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,1,1.0,"[Churrasqueira, Academia, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,0
3,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,NaN,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,0
4,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Condomínio ...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,296


In [212]:
df.to_excel('data/dados.xlsx', index=False)

## Identificar a geolocalização de cada endereço

In [206]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
geolocator = Nominatim(user_agent="myApp")

In [210]:
tqdm.pandas()

In [211]:
%%time
df['coord'] = df['address'].progress_apply(geocode).progress_apply(lambda location: (location.latitude, location.longitude))
df.head()

  6%|██▏                                    | 103/1826 [03:00<52:06,  1.81s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{}).
Traceback (most recent call last):
  File "c:\python\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "c:\python\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\python\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\python\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\python\lib\urllib\request.py", line 1389, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "c:\python\lib\urllib\request.py", line 1350, in do_open
    r = h.getresponse()
  File "c:\python\lib\http\client.py", line 1345, in getresponse
    response.begi

KeyboardInterrupt: 

In [205]:
df

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
0,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,1.0,"[Piscina, Churrasqueira, Condomínio fechado, A...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,225
1,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,0
2,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,1,1.0,"[Churrasqueira, Academia, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,0
3,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,NaN,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,0
4,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Condomínio ...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,296
...,...,...,...,...,...,...,...,...,...,...,...
1821,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,950,4650,2200
1822,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,3,3.0,[],"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,3500,8000,1250
1823,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,1,2.0,[],"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,870,4770,568
1824,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,NaN,1,NaN,"[Ar-condicionado, Elevador, Condomínio fechado...","Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,720,2690,224
